In [ ]:
import praw
import pandas as pd
import time
import spacy
from datetime import datetime

nlp = spacy.load('es_core_news_sm')

# Configura las credenciales de la API de Reddit
reddit = praw.Reddit(client_id='client_id',
                     client_secret='client_secret',
                     user_agent='user_agent')

# Especifica el subreddit
subreddit = reddit.subreddit('RepublicadeChile')

# Lista para almacenar los datos
data = []

def fetch_reddit_data(submission):
    # Convierte el timestamp de creación del post a una fecha legible
    post_date = datetime.fromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
    
    # Recopila los datos del hilo
    thread_data = {
        "Post ID": submission.id,
        "Título": submission.title,
        "Autor del Post": str(submission.author),
        "Puntuación del Post": submission.score,
        "URL": submission.url,
        "Número de comentarios": submission.num_comments,
        "Texto del Post": submission.selftext,
        "Fecha del Post": post_date,
        "Comentario ID": None,
        "Autor del Comentario": None,
        "Texto del Comentario": None,
        "Puntuación del Comentario": None,
        "Fecha del Comentario": None
    }
    data.append(thread_data)
    
    # Obtiene los comentarios
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        # Convierte el timestamp de creación del comentario a una fecha legible
        comment_date = datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
        
        comment_data = thread_data.copy()
        comment_data.update({
            "Comentario ID": comment.id,
            "Autor del Comentario": str(comment.author),
            "Texto del Comentario": comment.body,
            "Puntuación del Comentario": comment.score,
            "Fecha del Comentario": comment_date
        })
        data.append(comment_data)

# Inicializa la lista para almacenar los datos
data = []

# Itera sobre los hilos
contador = 0
# Límite de post que se van a extraer
for submission in subreddit.hot(limit=None):
    contador += 1
    fetch_reddit_data(submission)
    print(f"{contador}. Procesado post: {submission.id}")

# Crea el DataFrame
df = pd.DataFrame(data)

# Guarda el DataFrame en un archivo CSV
df.to_csv('post_hot.csv', index=False)

# Información adicional
print(f"\nNúmero total de filas en el DataFrame: {len(df)}")
print(f"Número de posts únicos: {df['Post ID'].nunique()}")
print(f"Número total de comentarios: {df['Comentario ID'].notna().sum()}")
print(df.head())


In [ ]:

#Lectura de archivos generados
df_new=pd.read_csv('post_new.csv')
df_top=pd.read_csv('post_top.csv')
df_hot=pd.read_csv('post_hot.csv')
df_controversial=pd.read_csv('post_controversial.csv')

#Identificación para cada set de datos en caso de que se quiera filtrar nuevamente
df_new['tipo_extraccion']=['new' for i in df_new['Post ID']]
df_top['tipo_extraccion']=['top' for i in df_top['Post ID']]
df_hot['tipo_extraccion']=['hot' for i in df_hot['Post ID']]
df_controversial['tipo_extraccion']=['controversial' for i in df_controversial['Post ID']]

dff=pd.concat([df_new, df_top, df_hot, df_controversial])
dff.reset_index(inplace=True, drop=True)

dff.to_csv('df_completo.csv', index=False)